# Differential Evolution (DE)

A population-based, stochastic optimiser that utilizes differences between individuals for evolution (hence the name).<br>

It requires only 3 parameters:<br>
 - Population size (also called number of individuals)<br> 
 - Differentiation constant: The weighting factor in creating a trial vector from adding the weighted difference between two vectors to a third one.<br> 
 - Crossover constant: Probability of mutation for each entry of the trial vector. This constant generates an interaction between classical crossover and mutation.<br>
 
Steps of the Algorithm:<br>
1. Initialise the population (such that they are well distributed in the design space. --> Design of Experiments)<br>
2. Loop through the following steps until maximum number of evaluations or generations is reached.<br>    
    a) Choose (random) individual and evaluate its fitness.<br>
    b) Create a trial individual by recombining three other individuals.   
    c) Select the better between trial and current individual.<br>
    
Due to its widespread use the algorithm is also implemented in Python standard libraries (e.g. SciPy).

## Import necessary libraries

In [ ]:
# -*- coding: utf-8 -*-
"""
Differential Evolution Algorithm for Structural Optimization 2 Tutorial

Chair of Computational Mechanics (Prof Duddeck)
Technische Universitaet Muenchen

Author: Gesche Fender

Adapted for Jupyter Lab by Arne Kaps
"""

import numpy as np
import random
from numpy.random import rand
import json
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D

## Objective function

In [ ]:
def Rosenbrock(X): 
    # Usually evaluated on [0,10]^2
    # Global minimum at (a,a^2) where f(x1,x2)=0
    x1, x2 = X
    a = 1
    b = 100     
    return (a-x1)**2 + b*(x2 - x1**2)**2

def Quadratic(X):
    # Can be evaluated on any search domain
    # Global minimum at (0,0) where f(x1,x2)=0
    x1, x2 = X
    return x1**2 + x2**2

def Ackley(X):
    # Usually evaluated on [-5,5]^2
    # Global minimum at (0, 0) where f(x1, x2) = 0
    x1, x2 = X
    return -20*np.exp(-0.2*np.sqrt(0.5*(x1**2+x2**2))) - np.exp(0.5*(np.cos(2*np.pi*x1)+np.cos(2*np.pi*x2))) + 20 + np.e

f = Quadratic

In [ ]:
# Plot the function to get an overview
delta = 0.01
x1 = np.arange(-5,5,delta)
x2 = np.arange(-5,5,delta)
X1,X2 = np.meshgrid(x1,x2)
Y = f((X1,X2))

fig = plt.figure(1)
ax = fig.gca(projection='3d')
ax.grid()
ax.set_xlabel("x2")
ax.set_ylabel("x1")
ax.invert_xaxis()
cntr = ax.plot_surface(X2,X1,Y,cmap=cm.viridis)
plt.show()

In [ ]:
delta = 0.1
x1 = np.arange(-10,10,delta)
x2 = np.arange(-10,10,delta)
X1,X2 = np.meshgrid(x1,x2)
Y = f((X1,X2))

fig = plt.figure(1)
ax = fig.gca(projection='3d')
ax.set_xlabel("x1")
ax.set_ylabel("x2")
cntr = ax.plot_surface(X1,X2,Y,cmap=cm.viridis)
plt.show()

 ## Initialisation methods

In [ ]:
def initialize():
    
        # Create population for initial generation:
        
        # Read user defined input parameters
        with open("fcoptexample_inifile.json", "r") as read_file:
            inidata = json.load(read_file)
            
        # Create initial Population and write parameter files  
        print('1. Starting to create population...')
        createIniPopulation(inidata)
        
def createIniPopulation(inifiledict):        
    #create population        
        
    L = inifiledict["bounds"]["lowerbounds"]
    H = inifiledict["bounds"]["upperbounds"]
    
    
    for j in range(int(inifiledict['numberindividuals'])):
        X = []
        for i in range(int(inifiledict['numberparameters'])):            
            #Create random individual inside bounds:
            X.append(L[i] + (H[i] - L[i])*rand()) #rand.uniform(L[i],H[i])
        # Write parameter file for each individual:
        writeinitialparameters(j,X, inifiledict)            
              
               
def initialsimulation(i):
    
    # Read parameters from parameterfile
    parameters = readparameters(i)
    X = parameters['parameters'][-1]
    
    #Calculate "simulation"-result for the parameter set
    result = f(X)
    
    #Write parameters to result file
    parameters['result'] = parameters['result']+[result]    
    writeparameters(parameters,i)    

## Simulation methods
This method simulates the trial of a single individual

In [ ]:
def simulation(individual, currentbest):
    
    currentparameters = readparameters(individual) #parameters of ith individual
    
    # Create list of all individuals different from the current one
    # Then pick three of those
    numberind = (currentparameters['numberindividuals']) #get number of individuals
    individuallist = [i for i in range(numberind) if i != individual]
    
    r = random.sample(individuallist,3)
    
    r0param = readparameters(r[0])
    r1param = readparameters(r[1])
    r2param = readparameters(r[2])
    
    # Create trial individual in which at least one parameter is changed
    trialvector = []
    
    rnd = np.floor(rand()*currentparameters['numberparameters'])
    
    
    for i in range(currentparameters['numberparameters']):
        if rand() < currentparameters['CR'] or rnd == i:  
            trialvector.append(r0param['parameters'][-1][i] + \
                               currentparameters['F']*(r1param['parameters'][-1][i] - r2param['parameters'][-1][i]))
        else:
            trialvector.append(currentparameters['parameters'][-1][i])

    
    # Verify boundary constraints
    #"bounds": {"lowerbounds": [0, 0], "upperbounds": [10, 10]}
    for i in range(currentparameters['numberparameters']):
        # If bounds are crossed, put random or to bounds instead? (problem dependent which is better)
        # Here it is put at random position inside the bounds
        if trialvector[i] < currentparameters['bounds']['lowerbounds'][i] or\
                trialvector[i] > currentparameters['bounds']['upperbounds'][i]:
            trialvector[i] = currentparameters['bounds']['lowerbounds'][i] + \
                        (currentparameters['bounds']['upperbounds'][i] - \
                         currentparameters['bounds']['lowerbounds'][i])*rand()

    
    # Execute simulation (here: evaluate the objective function)
    res = f(trialvector)
    
    oldparameters =  currentparameters['parameters'][-1] 
    oldresult = currentparameters['result'][-1]
    
    # Choose better one and write to resultfile
    if res <= oldresult:
        currentparameters['parameters'].append(trialvector)
        currentparameters['result'].append(res)    
        print(' Individual new result {}\n'.format(res))

        if res <= currentbest[-1]:
            currentbest.append(res)
            print('#####################################')    
            print('New Current Best:')
            print(res)
            print('#####################################\n') 
    
    else:
        currentparameters['parameters'].append(oldparameters)
        currentparameters['result'].append(oldresult)
        print(' Individual kept {}\n'.format(oldresult))

    writeparameters(currentparameters,individual)        
    

## Additional Functions 
They are needed for the file I/O and plotting of results.
Not so interesting for understanding the DE functionality.

In [ ]:
# Write initialisation file with all the input parameters
def writeinifile(numberevaluations, numberindividuals, numberparameters, lowerbounds, upperbounds, add_arguments, F, CR):
    inifiledata = { "numberevaluations": numberevaluations,
                "numberindividuals": numberindividuals,
                "numberparameters": numberparameters,
                "bounds": {"lowerbounds": lowerbounds,
                               "upperbounds": upperbounds},
                "add_arguments": add_arguments,
                "F": F,
                "CR": CR}
                
    with open("fcoptexample_inifile.json", "w") as write_file:
        json.dump(inifiledata, write_file)

# Write the initial positions of all individuals
def writeinitialparameters(i,X, inifiledict):
    print('This is X: {}\n'.format(X))
    paramfilenamejs = 'optresult_individual{}.json'.format(i)
    inifiledict["individual"] = i
    inifiledict["parameters"] = [list(X)]
    inifiledict["result"] = []
    with open(paramfilenamejs, "w") as write_file:
        json.dump(inifiledict, write_file)

# Write results for all individuals
def writeparameters(parameters,i):
    resultfilename = 'optresult_individual{}.json'.format(i)

    with open(resultfilename, "w") as write_file:
        json.dump(parameters, write_file, sort_keys=True, indent=4, separators=(',', ': '))

# Read data from file
def readparameters(i):
    with open('optresult_individual{}.json'.format(i), "r") as read_file:
            paramdata = json.load(read_file)
    return paramdata

# Print a summary of the important input parameters
def printinput(inidata):
    print('###############################\n')
    print('Following parameters were selected:\n')
    print('Population size: {}\n'.format(inidata['numberindividuals']))
    print('Differentiation constant F: {}\n'.format(inidata['F']))
    print('Crossover constant CR: {}\n'.format(inidata['CR']))
    print('###############################\n')
   
# Create a final convergence plot 
def plotconvergence(numberi, f, cr, kindofsim):
    
    for i in range(numberi):
        data = readparameters(i)
        
        plt.semilogy(data['result'])
        #plt.ylim(10e-8,10e6)
        plt.xlabel('Iterations per Individual')
        plt.ylabel('Quadratic(X)')
        plt.title(kindofsim+': '+str(numberi)+' individuals with F = '+str(f)+' and CR = '+str(cr))
    plt.show()

## Main Method

In [ ]:
def DE():
    
    # ENTER USER INPUT HERE: 

    numberindividuals = 10
    
    generations = 5#int(np.floor(numberevaluations/numberindividuals))
    
    numberevaluations = generations * numberindividuals #50
    
    F = 0.8

    CR = 0.9
    
    
    
    

    numberparameters = 2 # or can be derived from user input for bounds....

    lowerbounds = [0,0] # size must equal numberparameters

    upperbounds = [10,10] # size must equal numberparameters

    add_arguments = [] # if additional arguments are needed

    currentbest = [10e9]

    
    
    
    '''
    executing the optimization steps in order
    '''
    
    #Prepare: Change to right folder
    #os.chdir('path')
    
    
    #Step 1: Create file from user input (will be replaced by GUI output)
    writeinifile(numberevaluations, numberindividuals, numberparameters, lowerbounds, upperbounds, add_arguments, F, CR)
    
    # Load inidata so number of individuals and evaluations are known
    with open('fcoptexample_inifile.json', "r") as read_file:
            inidata = json.load(read_file)
    numberofindividuals = inidata['numberindividuals']
    #numberofevaluations = inidata['numberevaluations']
    printinput(inidata)
    
    #Step 2: Initialize Population
    initialize()
    
    #Step 3: Simulation of Initial Individuals
    
    for i in range(numberofindividuals):
        initialsimulation(i)
        
    #Step 4: Wait until all initial results are collected, then start optimization
    #       happens here automatically because of serial execution in loop
        
    #Step 5: Simulation of Individual
    #           -read three parametersets
    #           -create trial vector by combining those
    #           -simulation of trial vector
    #           -compare to old result
    #           -keep better
    
    #here everything is serial, but trying to emulate asynchronity by randomness 
    #TODO: implement convergence criterion, at the moment always running maximum evaluations
    '''
    for evaluation in range(numberevaluations):
        rndindividual = int(np.floor(rand()*numberofindividuals))
        print('{}.) Evaluating individual {}'.format(evaluation+1,rndindividual))
        simulation(rndindividual, currentbest)    
    simkind = 'Quasi-asynchronous DE'
    '''
    
    #alternatively quasi-generations (dynamic differential evolution:
    simkind = 'dynamic DE'
    for gen in range(generations):
        for individual in range(numberindividuals):
            print('evaluating individual {}'.format(individual))
            simulation(individual, currentbest)
    
    print('######################################')
    print('FINAL BEST: {}'.format(currentbest[-1]))
    plotconvergence(numberindividuals, F, CR, simkind)

if __name__ == "__main__":
    DE()

## Optional Exercises

1. Play around with the other objective functions. Plot them in their respective range. Which one do you think is the hardest to find an optimum for?

2. Play with the parameters of the simulations also switching between different objective functions. Notice how the behaviour of the algorithm changes.

3. There are different approaches to generating trial individuals. Right now random individuals are used. Add an option to use the current best one instead.

4. Right now, the script is terminating after a fixed number of generations / evaluations. Add the option to specify a termination value of the objective function or another convergence criterion (such as number of generations without changes).